In [1]:
# Config related imports
import config
from config import load_users_dataframe
unique_users = load_users_dataframe()
from config import load_networkx_friends
network_friends = load_networkx_friends()
from config import dump_ml_data

# Other imports
from tqdm import tqdm
import pandas as pd
import numpy as np

{'calculate': {'analysis': True,
               'friends': False,
               'network': True,
               'uniquetweets': True,
               'uniqueusers': True},
 'data': {'dates': ['2018-03-11', '2018-03-12', '2018-03-13'],
          'eventname': "Givenchy's Death",
          'phrases': ['givenchy%20death', 'givenchy%20passed%20away'],
          'starttime': 'Mar 12 08:20:00 -0500 2018'},
 'path': {'cwd': '/Users/taoguanyu/Workspace/uclresearchanalysis/data/givenchy',
          'ml': '/Users/taoguanyu/Workspace/uclresearchanalysis/data/givenchy/pickle',
          'networkx': {'all': '/Users/taoguanyu/Workspace/uclresearchanalysis/data/givenchy/pickle/networkx_all.dat',
                       'friends': '/Users/taoguanyu/Workspace/uclresearchanalysis/data/givenchy/pickle/networkx_friends.dat',
                       'potential': '/Users/taoguanyu/Workspace/uclresearchanalysis/data/givenchy/pickle/networkx_potential.dat'},
          'newcrawl': '/Users/taoguanyu/Workspace/uclr

In [2]:
unique_users.head()

,user,user_id,time_lapsed,followers_count,friends_count,user_created_at,user_statuses_count,user_listed_count,user_favourites_count,mention_and_reply,source_candidates,source_index,seed_index,generation,time_since_seed
0,davelackie,100766356,0.00,143119,4643,2992,181393,555,11938,[],"[3, 5, 6, 7, 10, 11, 14, 15, 19, 20, 24, 74, 8...",None,0,0,0
1,alexanderskhan,3186545203,0.15,753,3428,1041,12853,8,34186,[0],[0],0,0,1,0.15
2,consiglierela,4134992843,0.23,2161,396,855,46934,36,78797,[0],"[0, 12, 21, 32, 48, 748, 819, 972, 2357, 2778,...",0,0,1,0.23
3,ImpactPrincess,1955472014,0.63,242,642,1612,23179,55,28995,[0],"[0, 115, 577]",0,0,1,0.63
4,Damek0Masca,406301175,1.13,116,239,2317,2407,12,1058,[],[],None,4,0,0


In [3]:
# Settings
intervals = [60, 30, 15, 7]
parametersVector = ["tCurrent", "kIn", "kOut0", "t0", "kOut-1", "t-1", "kOutMax", "kOutMin", "kOutAverage", "tAverage", "nNodes", "deltaDays", "statusesCount", 
                    "followersCount", "favouritesCount", "friendsCount", "listedCount", "label"]

# Calculating in and out degrees
nodeInDegreeDict = network_friends.in_degree()
nodeOutDegreeDict = network_friends.out_degree()

# Self defined function for mean value calculation
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [4]:
def process_data(interval):
    # Prepare dataframe
    processed_dataframe = pd.DataFrame(columns=parametersVector)
    # Order the columns
    processed_dataframe = processed_dataframe[parametersVector]

    with tqdm(total=len(list(unique_users.iterrows()))) as pbar: 
        for index, user_row in unique_users.iterrows():
            if user_row['source_index'] is not None:
                source_candidates = user_row['source_candidates']
                source_first = source_candidates[0]
                source_first_time_lapsed = unique_users.iloc[source_first].time_lapsed
                start_bar = int(source_first_time_lapsed / interval) + 1

                label = []
                t0 = []
                t_1 = []
                kIn = []
                kOut0 = []
                kOut_1 = []
                kOutMax = []
                kOutMin = []
                kOutAverage = []
                nNodes = []
                tAverage = []

                bars = list(np.arange(start_bar * interval, 24 * 60, interval))

                for current_time in bars:
                    # all sources up to the current time
                    sources = [x for x in source_candidates if unique_users.iloc[x].time_lapsed <= current_time]
                    degreeList = [nodeOutDegreeDict[x] for x in sources]
                    timeList = [current_time - unique_users.iloc[x].time_lapsed for x in sources]

                    label.append(int(current_time >= user_row['time_lapsed']))
                    t0.append(round(timeList[0], 1))
                    t_1.append(round(timeList[-1], 1))
                    kIn.append(nodeInDegreeDict[index])
                    kOut0.append(nodeOutDegreeDict[sources[0]])
                    kOut_1.append(nodeOutDegreeDict[sources[-1]])
                    kOutMax.append(max(degreeList))
                    kOutMin.append(min(degreeList))
                    kOutAverage.append(round(mean(degreeList), 1))
                    nNodes.append(len(sources))
                    tAverage.append(round(mean(timeList), 1))

                processed_dataframe = processed_dataframe.append(pd.DataFrame({
                    'label': label,
                    'tCurrent': bars,
                    't0': t0,
                    't-1': t_1,
                    'kIn': kIn,
                    'kOut0': kOut0,
                    'kOut-1': kOut_1,
                    'kOutMax': kOutMax,
                    'kOutMin': kOutMin,
                    'kOutAverage': kOutAverage,
                    'nNodes': nNodes,
                    'tAverage': tAverage,
                    'deltaDays': user_row['user_created_at'],
                    'statusesCount': user_row['user_statuses_count'],
                    'followersCount': user_row['followers_count'],
                    'favouritesCount': user_row['user_favourites_count'], 
                    'friendsCount': user_row['friends_count'], 
                    'listedCount': user_row['user_listed_count']
                    }))
            pbar.update(1)
    return processed_dataframe

for interval in intervals:
    df = process_data(interval)
    print(df.head())
    dump_ml_data(df, interval)

  0%|          | 0/5011 [00:00<?, ?it/s]/Users/taoguanyu/anaconda2/envs/python35/lib/python3.5/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
100%|██████████| 5011/5011 [07:03<00:00,  7.85it/s]


  deltaDays favouritesCount followersCount friendsCount kIn kOut-1 kOut0  \
0      1041           34186            753         3428   1    183   183   
1      1041           34186            753         3428   1    183   183   
2      1041           34186            753         3428   1    183   183   
3      1041           34186            753         3428   1    183   183   
4      1041           34186            753         3428   1    183   183   

   kOutAverage kOutMax kOutMin label listedCount nNodes statusesCount    t-1  \
0        183.0     183     183     1           8      1         12853   60.0   
1        183.0     183     183     1           8      1         12853  120.0   
2        183.0     183     183     1           8      1         12853  180.0   
3        183.0     183     183     1           8      1         12853  240.0   
4        183.0     183     183     1           8      1         12853  300.0   

      t0  tAverage tCurrent  
0   60.0      60.0       60  
1 

100%|██████████| 5011/5011 [14:32<00:00,  3.89it/s]


  deltaDays favouritesCount followersCount friendsCount kIn kOut-1 kOut0  \
0      1041           34186            753         3428   1    183   183   
1      1041           34186            753         3428   1    183   183   
2      1041           34186            753         3428   1    183   183   
3      1041           34186            753         3428   1    183   183   
4      1041           34186            753         3428   1    183   183   

   kOutAverage kOutMax kOutMin label listedCount nNodes statusesCount    t-1  \
0        183.0     183     183     1           8      1         12853   30.0   
1        183.0     183     183     1           8      1         12853   60.0   
2        183.0     183     183     1           8      1         12853   90.0   
3        183.0     183     183     1           8      1         12853  120.0   
4        183.0     183     183     1           8      1         12853  150.0   

      t0  tAverage tCurrent  
0   30.0      30.0       30  
1 

100%|██████████| 5011/5011 [26:05<00:00,  2.13it/s]


  deltaDays favouritesCount followersCount friendsCount kIn kOut-1 kOut0  \
0      1041           34186            753         3428   1    183   183   
1      1041           34186            753         3428   1    183   183   
2      1041           34186            753         3428   1    183   183   
3      1041           34186            753         3428   1    183   183   
4      1041           34186            753         3428   1    183   183   

   kOutAverage kOutMax kOutMin label listedCount nNodes statusesCount   t-1  \
0        183.0     183     183     1           8      1         12853  15.0   
1        183.0     183     183     1           8      1         12853  30.0   
2        183.0     183     183     1           8      1         12853  45.0   
3        183.0     183     183     1           8      1         12853  60.0   
4        183.0     183     183     1           8      1         12853  75.0   

     t0  tAverage tCurrent  
0  15.0      15.0       15  
1  30.0   

100%|██████████| 5011/5011 [55:36<00:00,  1.08it/s]


  deltaDays favouritesCount followersCount friendsCount kIn kOut-1 kOut0  \
0      1041           34186            753         3428   1    183   183   
1      1041           34186            753         3428   1    183   183   
2      1041           34186            753         3428   1    183   183   
3      1041           34186            753         3428   1    183   183   
4      1041           34186            753         3428   1    183   183   

   kOutAverage kOutMax kOutMin label listedCount nNodes statusesCount   t-1  \
0        183.0     183     183     1           8      1         12853   7.0   
1        183.0     183     183     1           8      1         12853  14.0   
2        183.0     183     183     1           8      1         12853  21.0   
3        183.0     183     183     1           8      1         12853  28.0   
4        183.0     183     183     1           8      1         12853  35.0   

     t0  tAverage tCurrent  
0   7.0       7.0        7  
1  14.0   